# 4: Model Training

## Imports

In [1]:
from utils.model_utils import *

## Model Sanity Check

In [2]:
model = Chess_Model(bit_board_shape = (76, 8, 8), num_float_inputs = 4, channel_multiple = 4, concatenated_size = 588)
num_params = count_parameters(model)
print("Number of parameters in the model in millions:", round(num_params/(1e6), 4))

Number of parameters in the model in millions: 4.8321


In [3]:
print(model)

Chess_Model(
  (conv1): Conv2d(76, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(304, 76, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(76, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(304, 76, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (float_inputs_fc): Linear(in_features=4, out_features=512, bias=True)
  (fc1): Linear(in_features=588, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)


In [4]:
board_shape = (1, 76, 8, 8)
floats_shape = (1, 4)
input_bitboard = torch.tensor(np.random.rand(*board_shape), dtype = torch.float32)
input_floats = torch.tensor(np.random.rand(*floats_shape), dtype = torch.float32)
output = model(input_bitboard, input_floats)
output

tensor([[0.5101]], grad_fn=<SigmoidBackward0>)

## Model Training

In [5]:
LR = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
loss_func = nn.BCELoss()
NUM_HANGING_VALUES = 4
EPOCHS = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Detected Device: {device}")

Detected Device: cpu


In [6]:
train_generator = efficent_load_object("./Generators\\train_generator.pkl")
train_generator.__len__()

209557

In [7]:
val_generator = efficent_load_object("./Generators\\val_generator.pkl")
val_generator.__len__()

20931

In [8]:
test_generator = efficent_load_object("./Generators\\test_generator.pkl")
test_generator.__len__()

20974

In [9]:
model = train(train_generator, val_generator, test_generator, model, optimizer, loss_func, NUM_HANGING_VALUES, EPOCHS, device, log = 1, log_file = "./Training_Logs\\Training.txt", verbose = 1, val = True, early_callback=False, early_callback_epochs=None,
              checkpoint=True, epochs_per_checkpoint=1, break_after_checkpoint=False, checkpoint_filename="./Models\\PikeBot_Models\\PikeBot_checkpoint.pth")

______________________________________________________________
Epoch 0 Train Loss: 0.0109 | MSE: 0.2516 | MAE: 0.5015 | Accuracy: 0.5
Epoch 0 Val Loss: 0.0108 | MSE: 0.2502 | MAE: 0.5001  | Accuracy: 0.4688
Epoch 1: Saving checkpoint...
______________________________________________________________
Epoch 1 Train Loss: 0.0108 | MSE: 0.2475 | MAE: 0.4974 | Accuracy: 0.5156
Epoch 1 Val Loss: 0.0108 | MSE: 0.2484 | MAE: 0.4982  | Accuracy: 0.5312
Epoch 2: Saving checkpoint...
______________________________________________________________
Epoch 2 Train Loss: 0.0106 | MSE: 0.244 | MAE: 0.4938 | Accuracy: 0.6719
Epoch 2 Val Loss: 0.0107 | MSE: 0.2468 | MAE: 0.4964  | Accuracy: 0.5625
Epoch 3: Saving checkpoint...
______________________________________________________________
Epoch 3 Train Loss: 0.0105 | MSE: 0.2408 | MAE: 0.4904 | Accuracy: 0.7031
Epoch 3 Val Loss: 0.0107 | MSE: 0.2454 | MAE: 0.4948  | Accuracy: 0.5938
Epoch 4: Saving checkpoint...
____________________________________________

## Saving Model

In [10]:
model

Chess_Model(
  (conv1): Conv2d(76, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(304, 76, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(76, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(304, 304, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(304, 76, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (float_inputs_fc): Linear(in_features=4, out_features=512, bias=True)
  (fc1): Linear(in_features=588, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [11]:
temp_state_path = "./Models\\PikeBot_Models\\temp_state_dict.pth"
torch.save(model.state_dict(), temp_state_path)
model = Chess_Model(bit_board_shape = (76, 8, 8), num_float_inputs = 4, channel_multiple = 4, concatenated_size = 588)
model.load_state_dict(torch.load(temp_state_path))
save_model(model, model_filename = "./Models\\PikeBot_Models\\PikeBot.pth", onnx_filename= "./Models\\PikeBot_Models\\PikeBot.onnx", bitboard_input_shape = (1, 76, 8, 8), hanging_values_input_shape = (1, 4), opset_version=11, device="cpu")

Model saved successfully!
